# Generate the Driver Profile (Enriched Zone: driver) using LLMs
- Create a Qualitative data analysis for Driver Habits
    - We have found some patterns that we want to dive deeper into:
      - trunk space
      - driving speed
      - hours worked
      - preferred pickup locations
      - average trip distance
      - crossing state lines
      - speaks spanish
      - vechicle cleanliness
      - vechile temperature
      - maximum passengers
      - conversation with customer
      - music playing
      - distracted driver
      - target pay

## Score all items in batches
- Find all records that have not been scored
- Score in a batch (we can do up to 10,000)
- The LLM temperature, max_output_tokens, top_p and top_k parameters have been set (locked for a deterministic value)
- Repeat until done

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

In [ ]:
%%bigquery

-- OPTIONAL: Reset all the fields to null
-- If you need to reset you data back to fresh data run the stored procedure: CALL `data-analytics-demo-bhp0hynrof.rideshare_llm_curated.sp_reset_demo`();
/*
UPDATE `data-analytics-demo-bhp0hynrof.rideshare_llm_enriched.customer_review` AS customer_review
   SET extracted_driver_theme_json = NULL,
       extracted_driver_theme = NULL
 WHERE TRUE;
*/

In [ ]:
# need quotes around each category; otherwise, we get part of the category text
prompt = """
Classify the text as one of the following categories:
- "trunk space small"
- "trunk space large"
- "driving too fast"
- "driving too slow"
- "driver speaks spanish"
- "driver does not speak spanish"
- "clean car"
- "dirty car"
- "car too hot"
- "car too cold"
- "driver likes conversation"
- "driver likes no conversation"
- "driver likes music"
- "driver likes no music"
- "distracted driver"

Text:
"""

In [ ]:
# Proces in batches
batch_size = 100

# Set the parameters so we are more deterministic and less creative/random responses
llm_temperature = 1
llm_max_output_tokens = 50
llm_top_p = 0
llm_top_k = 1

update_sql="""
UPDATE `data-analytics-demo-bhp0hynrof.rideshare_llm_enriched.customer_review` AS customer_review
  SET extracted_driver_theme_json = child.ml_generate_text_result
  FROM (SELECT *
          FROM ML.GENERATE_TEXT(MODEL`data-analytics-demo-bhp0hynrof.rideshare_llm_enriched.cloud_ai_llm_v1`,
              (SELECT trip_id,
                      customer_id,
                      CONCAT(\"\"\"{prompt}\"\"\",customer_review_text) AS prompt
                FROM `data-analytics-demo-bhp0hynrof.rideshare_llm_enriched.customer_review`
                WHERE (extracted_driver_theme_json IS NULL
                       OR
                       JSON_VALUE(extracted_driver_theme_json, '$.predictions[0].content') IS NULL
                       )
                  AND customer_review_text IS NOT NULL
                LIMIT {batch_size}),
              STRUCT(
                {llm_temperature} AS temperature,
                {llm_max_output_tokens} AS max_output_tokens,
                {llm_top_p} AS top_p,
                {llm_top_k} AS top_k
                ))
  ) AS child
WHERE customer_review.trip_id = child.trip_id
  AND customer_review.customer_id = child.customer_id;
  """.format(batch_size = batch_size,
             prompt = prompt,
             llm_temperature = llm_temperature,
             llm_max_output_tokens = llm_max_output_tokens,
             llm_top_p = llm_top_p,
             llm_top_k = llm_top_k)

print("SQL: {update_sql}".format(update_sql=update_sql))


In [ ]:
# Score while records remain
# score in groups of batch_size records (we can do up to 10,000 at a time)
import time

done = False
original_record_count = 0
displayed_first_sql = False

while done == False:
  # Get the count of records to score
  sql = """
        SELECT COUNT(*) AS cnt
          FROM `data-analytics-demo-bhp0hynrof.rideshare_llm_enriched.customer_review`
         WHERE (extracted_driver_theme_json IS NULL
                OR
                JSON_VALUE(extracted_driver_theme_json, '$.predictions[0].content') IS NULL
                )
           AND customer_review_text IS NOT NULL;
        """

  df_record_count = client.query(sql).to_dataframe()
  cnt = df_record_count['cnt'].head(1).item()
  if displayed_first_sql == False:
    original_record_count = cnt
    displayed_first_sql = True

  print("Remaining records to process: ", cnt, " out of", original_record_count, " batch_size: ", batch_size)

  if cnt == 0:
    done = True
  else:
    # https://github.com/googleapis/python-bigquery/tree/master/samples
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(update_sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))

    while query_job.state != "DONE":
      time.sleep(5)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))


## Parse the LLM JSON results

In [ ]:
%%bigquery

UPDATE `data-analytics-demo-bhp0hynrof.rideshare_llm_enriched.customer_review` AS customer_review
   SET extracted_driver_theme = JSON_VALUE(extracted_driver_theme_json, '$.predictions[0].content')
 WHERE extracted_driver_theme_json IS NOT NULL
   AND extracted_driver_theme IS NULL;

In [ ]:
%%bigquery

SELECT trip_id, customer_review_text, extracted_driver_theme
  FROM `data-analytics-demo-bhp0hynrof.rideshare_llm_enriched.customer_review`
 WHERE extracted_driver_theme_json IS NOT NULL
   AND extracted_driver_theme IS NOT NULL
LIMIT 20;

In [ ]:
%%bigquery

SELECT extracted_driver_theme, count(*) AS cnt
  FROM `data-analytics-demo-bhp0hynrof.rideshare_llm_enriched.customer_review`
 WHERE extracted_driver_theme_json IS NOT NULL
   AND extracted_driver_theme IS NOT NULL
GROUP BY 1;
